In [1]:
import pandas as pd
import requests
import os
from google.cloud import bigquery

In [2]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = 'secret.json'

bq_client = bigquery.Client()

[API Docs](https://dev.socrata.com/foundry/data.cityofnewyork.us/ic3t-wcy2)  
[OpenData](https://data.cityofnewyork.us/Housing-Development/DOB-Job-Application-Filings/ic3t-wcy2)  
[Metadata](https://docs.google.com/spreadsheets/d/1TGe-YvR-rqV3sTm2DVl9icoiypo3ug-GDdP1ULdo1_s/edit#gid=169410431)

In [3]:
ACRIS=f'https://data.cityofnewyork.us/resource/bnx9-e6tj.json?$'

In [4]:
acris_count=f'https://data.cityofnewyork.us/resource/bnx9-e6tj.json?$select=count(1) as cnt'
cnt_req=requests.get(acris_count).json()
nrows=int(cnt_req[0]['cnt'])

In [5]:
limit=10**6
offset=0

offs=list(range(0, nrows, limit))

reqs=[f'{ACRIS}limit={limit}&$offset={o}&$order=recorded_datetime' for o in offs]

In [9]:
table_id='pluto-panel.real_estate.raw_acris_master2'

job_config = bigquery.LoadJobConfig(
        autodetect=True,
        write_disposition='WRITE_APPEND'

        )

In [13]:
prog=8
for r in reqs[8:]:
    print(f'reading {prog} / {len(reqs)}: {r}')
    d=requests.get(r).json()
    print('making dataframe')
    d2=pd.DataFrame(d)
    print('loading to bigquery')
    job = bq_client.load_table_from_dataframe(
            d2, table_id, job_config=job_config)
    job.result()
    prog+=1
    
    

reading 8 / 16: https://data.cityofnewyork.us/resource/bnx9-e6tj.json?$limit=1000000&$offset=8000000&$order=recorded_datetime
making dataframe
loading to bigquery
reading 9 / 16: https://data.cityofnewyork.us/resource/bnx9-e6tj.json?$limit=1000000&$offset=9000000&$order=recorded_datetime
making dataframe
loading to bigquery
reading 10 / 16: https://data.cityofnewyork.us/resource/bnx9-e6tj.json?$limit=1000000&$offset=10000000&$order=recorded_datetime
making dataframe
loading to bigquery
reading 11 / 16: https://data.cityofnewyork.us/resource/bnx9-e6tj.json?$limit=1000000&$offset=11000000&$order=recorded_datetime
making dataframe
loading to bigquery
reading 12 / 16: https://data.cityofnewyork.us/resource/bnx9-e6tj.json?$limit=1000000&$offset=12000000&$order=recorded_datetime
making dataframe
loading to bigquery
reading 13 / 16: https://data.cityofnewyork.us/resource/bnx9-e6tj.json?$limit=1000000&$offset=13000000&$order=recorded_datetime
making dataframe
loading to bigquery
reading 14 / 1

In [14]:
table = bq_client.get_table(table_id)  # Make an API request.
print(
    "Loaded {} rows and {} columns to {}".format(
        table.num_rows, len(table.schema), table_id
    )
)

Loaded 8600757 rows and 14 columns to pluto-panel.real_estate.raw_acris_master2
